In [3]:
import argparse
import glob
import os
import torch
import re
import pandas as pd
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader

from config import get_evaluate_config
from dataset import data_loader
from eval.post_process import *
from neural_methods.model.PhysNet import PhysNet_padding_Encoder_Decoder_MAX
from neural_methods.model.TS_CAN import TSCAN
from neural_methods.model.DeepPhys import DeepPhys
from eval.post_process import *
from collections import OrderedDict
import random
import numpy as np

RANDOM_SEED = 100
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
g = torch.Generator()
g.manual_seed(RANDOM_SEED)

ModuleNotFoundError: No module named 'torch'

In [ ]:
def add_args(parser):
    """Adds arguments for parser."""
    parser.add_argument('--config_file', required=False,
                        default="configs/UBFC_TSCAN_EVALUATION.yaml", type=str, help="The name of the model.")
    parser.add_argument(
        '--device',
        default=-1,
        type=int,
        help="An integer to specify which gpu to use, -1 for cpu.")
    parser.add_argument(
        '--model_path', required=False, type=str)
    parser.add_argument('--batch_size', default=1, type=int)
    parser.add_argument('--test_data_path', default='/Users/zhangxiaoyu/Documents/Toolbox/DeepLearningProject/UBFC-Valid/subject1', required=False,
                        type=str, help='The path of the data directory.')
    parser.add_argument('--log_path', default=None, type=str)
    return parser

In [ ]:
parser = argparse.ArgumentParser()
parser = add_args(parser)
parser = data_loader.BaseLoader.BaseLoader.add_data_loader_args(parser)
args = parser.parse_args()
args.test_data_path = '/Users/zhangxiaoyu/Documents/Toolbox/DeepLearningProject/UBFC-Valid/subject1'
args.model_path = '/Users/zhangxiaoyu/Documents/Toolbox/DeepLearningProject/rPPG-Toolbox-1/PreTrainedModels/UBFC_SizeW72_SizeH72_ClipLength180_DataTypeNormalized_Standardized_LabelTypeNormalized/tscan_train_ubfc.pth'

# forms configurations.
config = get_evaluate_config(args)
print('=== Printing Config ===')

In [ ]:
loader = data_loader.UBFCLoader.UBFCLoader
